In [6]:
# These may be needed on new environments
!pip install --upgrade pip
!pip install onnx
!pip install onnxruntime 
!pip install numpy --upgrade
!pip install cntk --upgrade 
!pip install keras --upgrade


     |################################| 75.1MB 10.8MB/s eta 0:00:01   |##                              | 5.0MB 2.4MB/s eta 0:00:30     |###                             | 7.7MB 3.7MB/s eta 0:00:19     |###                             | 8.0MB 3.7MB/s eta 0:00:19     |###                             | 8.9MB 3.7MB/s eta 0:00:19     |####                            | 9.4MB 3.7MB/s eta 0:00:18     |#####                           | 12.2MB 3.8MB/s eta 0:00:17     |#####                           | 13.4MB 3.8MB/s eta 0:00:17     |######                          | 14.4MB 3.8MB/s eta 0:00:16     |############                    | 30.3MB 2.6MB/s eta 0:00:18     |##################              | 44.4MB 6.0MB/s eta 0:00:06     |###################             | 45.6MB 6.0MB/s eta 0:00:05     |###################             | 46.8MB 12.5MB/s eta 0:00:03     |####################            | 48.7MB 12.5MB/s eta 0:00:03     |#####################           | 49.5MB 12.5MB/s eta 0:00:03     |#######

In [56]:
# Set keras to use cntk back-end
!rm /root/.keras/keras.json
#!echo '{"floatx": "float32","image_data_format": "channels_last","backend": "cntk","epsilon": 1e-07}' > /root/.keras/keras.json

In [57]:
# Import libraries
import cntk
import onnx
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import onnxruntime as rt

In [58]:
# Training Set
# Vectors - [height, weight, shoe_size]
X = np.array([[181, 80, 44], [177, 70, 43], [160, 60, 38],
     [154, 54, 37], [166, 65, 40], [190, 90, 47],
     [175, 64, 39], [177, 70, 40], [159, 55, 37],
     [171, 75, 42], [181, 85, 43]])

# Labels - 1 per vector
Y = np.array([0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0])

# create model
model = Sequential()
model.add(Dense(60, input_dim=3, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# train model
model.fit(X, Y, epochs=100)

Epoch 1/100
11/11 [==============================] - 0s - loss: 8.6958 - acc: 0.4545
Epoch 2/100
11/11 [==============================] - 0s - loss: 8.6958 - acc: 0.4545
Epoch 3/100
11/11 [==============================] - 0s - loss: 8.6958 - acc: 0.4545
Epoch 4/100
11/11 [==============================] - 0s - loss: 8.6958 - acc: 0.4545
Epoch 5/100
11/11 [==============================] - 0s - loss: 8.6958 - acc: 0.4545
Epoch 6/100
11/11 [==============================] - 0s - loss: 8.6958 - acc: 0.4545
Epoch 7/100
11/11 [==============================] - 0s - loss: 8.6958 - acc: 0.4545
Epoch 8/100
11/11 [==============================] - 0s - loss: 8.6958 - acc: 0.4545
Epoch 9/100
11/11 [==============================] - 0s - loss: 8.6958 - acc: 0.4545
Epoch 10/100
11/11 [==============================] - 0s - loss: 8.6958 - acc: 0.4545
Epoch 11/100
11/11 [==============================] - 0s - loss: 8.6958 - acc: 0.4545
Epoch 12/100
11/11 [==============================] - 0s - loss

In [76]:
!pip install protobuf --upgrade
!pip install tensorflow --upgrade
import keras2onnx
print(model.summary())
print(model.name)
onnx_model = keras2onnx.convert_keras(model, model.name)

Requirement already up-to-date: protobuf in /root/anaconda3/envs/cntk-py35/lib/python3.5/site-packages (3.9.1)
Requirement already up-to-date: tensorflow in /root/anaconda3/envs/cntk-py35/lib/python3.5/site-packages (1.14.0)


TypeError: __new__() got an unexpected keyword argument 'serialized_options'

In [59]:
# save the model in ONNX format
model.save('/mount/gender_classifier.n5')

In [71]:
# Test the ONNX model using the ONNX runtime
sess = rt.InferenceSession("/mount/gender_classifier.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
X_test = np.array([160,60,43])
pred_onx = sess.run([label_name], {input_name: X_test.astype(np.float32)})[0]
pred_onx

RuntimeError: [ONNXRuntimeError] : 1 : GENERAL ERROR : Load model from /mount/gender_classifier.onnx failed:ModelProto does not have a graph.

In [69]:
!pip install numpy==1.16.1
!pip install setuptools --upgrade
!mmconvert -sf keras -iw /mount/gender_classifier.n5 -df cntk -om /mount/gender_classifier.onnx

  Using cached https://files.pythonhosted.org/packages/b2/86/095d2f7829badc207c893dd4ac767e871f6cd547145df797ea26baea4e2e/setuptools-41.2.0-py2.py3-none-any.whl
  Found existing installation: setuptools 27.2.0
ERROR: Cannot remove entries from nonexistent file /root/anaconda3/envs/cntk-py35/lib/python3.5/site-packages/easy-install.pth
Using TensorFlow backend.





2019-09-15 12:48:41.815880: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-09-15 12:48:41.820704: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2808000000 Hz
2019-09-15 12:48:41.821084: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x4832420 executing computations on platform Host. Devices:
2019-09-15 12:48:41.821130: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2019-09-15 12:48:41.846020: W tensorflow/compiler/jit/mark_fo

In [70]:
# Prepare and check the saved model for serialization
onx = onnx.load('/mount/gender_classifier.onnx')
print(onx)

ir_version: 2
producer_name: "\n\007version\022\006\010\002\020\003(\003"



In [ ]:
# Store the model in MarkLogic
import requests
from requests.auth import HTTPDigestAuth

try:
    url = 'http://aidata:8011/v1/documents'
    header = {"Accept": "application/json"}
    params = 'uri=/mount/gender_classifier.onnx'
    
    response = (requests.put(url+"?"+params, auth=HTTPDigestAuth('admin', 'admin'), headers=header, data=onx.SerializeToString()))
    print(response)
except Exception as e:
    raise e